In [1]:
import cv2
import keras
from threading import Thread
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
print('cv2: ' + cv2.__version__ + '\n' + 'matplotlib: ' + matplotlib.__version__)

Using TensorFlow backend.


cv2: 3.4.3
matplotlib: 2.2.3


In [2]:
try:
    model = keras.models.load_model('model.h5')
except:
    print('error inloading model')

In [3]:
# comman functions required for cv2
class VideoStream:
    
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):
        Thread(target=self.update, args=()).start()
        return self
    
    def update(self):
        while True:
            if self.stopped:
                return
            (self.grabbed, self.frame) = self.stream.read()
            
    def read(self):
        # Return the latest frame
        return self.frame
    
    def stop(self):
        self.stream.release()
        cv2.destroyAllWindows()
        self.stopped = True

In [4]:
def reshape(image):
    try:
        image = image.reshape(1, 28, 28)
        print('---before image', image.shape)
        #plt.imshow(image[0])
        #plt.show()
        image = image.reshape(1, 28, 28, 1).astype('uint8')
        return image
    except Exception as e: 
        print('Exception in predict:'+ e)

In [5]:
def changeImage(path):
    im = cv2.imread(path)
    morph = im.copy()

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    morph = cv2.morphologyEx(morph, cv2.MORPH_CLOSE, kernel)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # split the gradient image into channels
    image_channels = np.split(np.asarray(morph), 3, axis=2)

    channel_height, channel_width, _ = image_channels[0].shape

    # apply Otsu threshold to each channel
    for i in range(0, 3):
        _, image_channels[i] = cv2.threshold(image_channels[i], 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
        image_channels[i] = np.reshape(image_channels[i], newshape=(channel_height, channel_width, 1))

    # merge the channels
    image_channels = np.concatenate((image_channels[0], image_channels[1], image_channels[2]), axis=2)
    image_channels = cv2.cvtColor( image_channels, cv2.COLOR_BGR2GRAY )
    image_channels[image_channels==255] = 200
    image_channels[image_channels==0]   = 100
    image_channels[image_channels==100] = 255
    image_channels[image_channels==200] = 0
    # save the denoised image
    cv2.imwrite(path, image_channels)
    return image_channels

In [6]:
def predict(path):
    img = changeImage(path)
    #img = cv2.imread(path,0)
    #print('----before reshape', img.shape)
    cv2.imshow('noise cancellation', img)
    img = img.reshape(1, 28, 28, 1).astype('uint8')
    predicted = model.predict(img)
    predicted = np.argmax(predicted, axis=None, out=None)
    print('predicted digit:', predicted)

In [13]:
def chnagePixelAndPredict(frame):
    frame[frame<97] = 0
    frame[frame>96] = 255 
    img = frame
    cv2.imshow('noise cancellation', img)
    img = img.reshape(1, 28, 28, 1).astype('uint8')
    predicted = model.predict(img)
    predicted = np.argmax(predicted, axis=None, out=None)
    print('predicted digit:', predicted)
    return frame

In [16]:
# Init video stream
vs = VideoStream(src=0).start()

In [17]:
'''
save image frame in black and white, then read its pixel value and predict the result.
'''
frame_count=0
while True:
    frame = vs.read()
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) == 27: 
            break  # esc to quit
    dim = (28, 28)
    frame = cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)
    frame = cv2.cvtColor( frame, cv2.COLOR_BGR2GRAY )
    frame_count+=1
    try:
        # Only run every 15 framaes
        if frame_count%15==0:
            # Save the image as the fist layer of inception is a DecodeJpeg
            frame = chnagePixelAndPredict(frame)
            cv2.imwrite("current_frame.jpg",frame)
            #predict("current_frame.jpg")
            frame_count = 0
            #_ = system('clear') 
            #print('--------frame before reshape', frame.shape)
            #frame = reshape(frame)
            #print('--------frame before reshape', frame.shape)
            #print(exit)
            '''predicted = model.predict(frame)
            predicted = np.argmax(predicted, axis=None, out=None)
            print('predicted digit:', predicted)         '''
    except Exception as e: 
        print('Exception in predict:', e)
        print(exit)
        vs.stop()
vs.stop()

frame before [[132 133 132 130 128 127 129 131 133 136 137 138 140 141 143 149 152 156
  159 162 163 178 214 208 109  35  20  21]
 [132 132 132 130  75  63 122 129 131 135 137 137 140 141 141 145 150 156
  159 162 162 176 214 204  60  40  38  56]
 [131 131 131 128  79  69  85 129 131 131 134 135 137 138 139 141 147 152
  157 161 162 173 213 142  52 110  87  74]
 [131 130 132 131 133  96  95 123 127 132 133 136 137 140 144 145 148 152
  155 161 163 173 190 112 104 137 103  42]
 [130 129 131 131 127  48  54  65 121 131 133 136 137 139 141 143 145 150
  154 159 162 169 138  99 124  89  94  45]
 [126 123 122 125  98  74 118 126 129 131 133 134 136 137 140 141 146 147
  154 158 161 162 123 126 115  71  77  19]
 [122 120 120 122 123 123 125 128 129 130 131 133 134 136 138 140 142 147
  151 157 162 162 160 132 111  68  39  22]
 [120 121 121 121 122 122 124 126 127 129 129 131 132 135 137 139 142 146
  150 156 160 162 150 136 113  65  48  46]
 [123 121 121 118 119 120 120 124 124 126 128 129 1

frame before [[130 130 128 127 128 129 129 132 132 133 135 135 137 139 142 146 149 153
  156 159 161 167 213 206  77  29  19  23]
 [129 128 127 126 125 126 127 129 131 133 134 135 135 138 141 144 149 152
  156 160 161 166 213 185  49  39  39  58]
 [128 127 126 113 110 123 126 127 129 131 133 134 135 136 139 143 147 152
  155 158 160 164 211 112  59 112  84  69]
 [128 126 124  44  35 102 123 125 127 128 131 132 134 135 136 139 145 149
  153 157 159 162 165 110 111 137  89  40]
 [127 127 126  97  85  70 125 125 124 126 128 130 131 133 135 137 141 147
  151 156 159 161 114 104 122  89  88  38]
 [126 126 127 129  90  88 109 112 126 126 130 132 134 137 138 140 142 147
  151 156 160 160 116 126 107  69  69  18]
 [125 126 126 117  36  41  61 109 125 127 128 130 132 133 135 138 139 146
  149 154 156 156 146 132 102  62  35  22]
 [118 117 119  92  71 114 119 123 124 126 128 128 130 132 134 137 141 143
  149 153 160 157 144 136 102  59  45  45]
 [116 116 119 118 117 119 121 122 124 122 125 126 1

frame before [[133 132 132 131 131 132 134 133 134 135 136 138 140 143 145 148 152 157
  159 161 162 206 214 124  38  21  18  31]
 [131 130 130 130 131 131 132 133 134 134 135 137 139 142 145 148 152 156
  159 161 162 203 213  78  43  31  49  54]
 [132 130 129 127 128 128 130 132 133 134 135 136 138 140 144 147 151 155
  158 159 161 200 167  51  96  88  74  52]
 [130 128 127 115 123 124 128 129 131 132 134 135 137 138 143 146 151 153
  157 160 160 191 120  97 145  95  51  47]
 [129 128 105  33  69 122 125 127 131 132 133 134 135 136 139 143 147 152
  156 158 159 151  89 126 109 108  43  28]
 [129 128 112 103  43 116 126 126 128 129 130 132 133 135 136 140 145 149
  154 157 159 131 113 123  74  92  21  19]
 [128 128 128 131  55 122 107 125 127 128 132 134 136 138 138 142 146 149
  154 159 160 149 136 119  72  45  24  26]
 [127 128 126  88  35  52  84 124 126 128 130 132 134 135 136 138 143 148
  152 156 155 152 142 120  68  45  38  50]
 [120 119 120  72  94 120 122 125 126 128 128 130 1

frame before [[134 134 135 134 135 135 136 137 138 140 140 142 146 149 152 155 157 161
  162 167 220 217 204  55  28  16  29  22]
 [134 134 134 135 135 134 135 136 136 138 139 141 145 148 151 155 158 161
  162 163 216 218 166  51  36  37  59  44]
 [133 132 132 132 133 133 134 135 136 137 138 139 143 147 149 153 156 160
  161 161 211 214  98  73 103  82  64  46]
 [131 130 129 130 131 132 133 134 135 135 137 139 141 144 148 151 156 159
  160 161 206 169 106 133 128  67  49  42]
 [130 128 126 125 127 129 132 133 134 135 136 137 138 143 146 151 154 157
  159 160 202 111 109 127 116  77  25  41]
 [130 113  50  96 124 126 129 131 133 134 135 135 136 140 143 148 152 156
  158 159 176 113 126  99  82  56  19  25]
 [129 101  80  39 123 126 127 127 129 131 132 134 135 137 141 145 150 154
  158 160 167 134 138  98  61  29  19  34]
 [129 130 131  52 128 119 125 127 130 132 134 137 139 139 141 146 150 155
  158 159 179 149 141  97  55  41  43  47]
 [130 130 102  42  51  81 126 128 129 131 132 134 1

frame before [[132 132 133 132 134 134 135 136 137 137 138 141 143 146 150 154 156 159
  161 162 201 216 144  41  24  17  28  15]
 [134 133 132 132 132 132 134 134 134 136 138 139 142 146 148 152 156 158
  161 161 192 217 100  46  31  44  52  36]
 [132 130 131 129 130 132 134 133 134 136 137 138 141 143 147 151 154 157
  160 161 186 185  59  91  92  77  53  45]
 [130 130 127 127 127 130 131 133 133 135 135 136 140 141 145 150 153 157
  158 160 182 124 103 150 103  56  48  37]
 [129 126 118 123 125 126 129 131 132 133 134 136 136 140 143 148 151 155
  158 159 162  87 123 118 115  50  26  42]
 [128 107  33  66 123 125 127 129 131 132 134 134 135 137 142 145 150 153
  158 157 147 109 129  81  88  36  17  33]
 [128 114  99  45 111 124 125 126 128 128 131 133 135 136 139 144 147 152
  157 158 154 136 132  79  53  21  19  34]
 [128 127 131  59 117 105 123 126 129 131 133 135 138 138 139 144 147 152
  157 157 158 146 131  78  46  35  44  34]
 [128 126  99  32  47  79 122 126 127 128 131 132 1

...

frame before [[254 254 254 254 254 243 214 182 143 127 124 123 123 123 123 125 125 127
  128 129 129 130 129 129 129 130 132 132]
 [254 254 254 254 254 248 223 191 152 127 123 123 122 123 122 123 124 125
  127 128 130 130 131 130 130 130 131 132]
 [254 254 254 254 254 253 231 200 163 130 124 123 123 122 122 122 124 125
  125 127 128 128 130 130 130 131 131 132]
 [254 254 254 254 254 254 239 208 174 135 126 124 123 122 122 120 122 123
  124 126 126 127 128 128 131 131 132 132]
 [254 254 254 254 254 254 246 218 183 143 126 124 123 122 122 120 121 121
  122 124 124 126 128 128 129 130 130 131]
 [254 254 254 254 254 254 249 223 191 153 126 123 122 121 121 120 121 122
  122 123 123 124 127 128 128 129 129 130]
 [254 254 254 254 254 254 249 226 197 161 128 122 121 121 120 120 120 121
  122 123 123 123 124 127 128 128 129 130]
 [254 254 254 254 254 254 251 229 200 167 132 121 120 120 120 119 120 119
  121 121 122 123 123 124 125 127 128 130]
 [254 254 254 254 254 254 250 230 202 173 137 119 1

frame before [[242 245 247 248 246 244 241 237 233 229 224 217 208 206 169 141 129 182
  173 141 175 171 168 164 113  16  10  26]
 [247 249 251 251 249 246 242 238 233 229 224 218 209 204 194 184 179 191
  157 175 176 172 169 165  84  23  36  47]
 [254 253 253 253 250 247 243 238 234 230 224 190 168 177 187 184 185 186
  177 179 178 174 169 160  60  76  57  41]
 [254 254 254 254 253 249 244 239 234 230 225 166 144 142 139 151 159 179
  179 178 176 173 169 128  73 105  61  16]
 [254 254 254 254 253 249 244 241 234 230 227 198 133 130 121 128 138 155
  164 168 174 172 163  91  90  65  47  13]
 [254 254 254 253 252 247 243 239 233 229 228 195 128 131 133  99 150 155
  146 143 153 166 140  79  85  48  20   8]
 [254 254 254 251 247 243 239 235 231 227 227 186 124 131 134  38  23  64
  113 155 133 133 119 101  83  44  27  35]
 [253 252 250 247 244 239 235 231 227 223 222 175 120 128 130 193 100  77
   27  76 154 138 131 104  87  40  44  58]
 [249 248 246 243 239 234 229 225 220 218 217 163 1

frame before [[237 241 242 243 242 240 236 233 229 224 219 211 204 202 166 135 125 177
  169 136  81  27  52  36  26  22  16   5]
 [243 245 246 246 245 242 237 233 229 224 220 212 205 199 189 179 174 186
  152 169  77  79  42  26  28  27  22  10]
 [252 251 250 249 246 243 239 234 230 226 219 185 164 172 183 180 180 181
  172 170  94 103  48  10  19  26  27  18]
 [254 254 254 253 250 245 239 235 230 225 220 162 140 138 135 147 154 174
  174 152  92  66  27   7  20  10   7  14]
 [253 254 254 252 250 245 241 237 230 225 222 193 129 126 116 124 133 150
  159 113  95  56  25  33  34   6  10  23]
 [253 253 252 250 248 242 238 234 229 225 222 191 125 126 128  95 146 150
  141 108 106  60  38  63  46  16  13  38]
 [252 251 250 247 243 239 235 231 226 221 222 181 120 127 131  35  20  61
  108 111 111  58  30  31  30  30  35  61]
 [249 247 246 243 240 235 231 226 221 218 218 170 115 123 125 190  96  73
   24  88 106  62  27  14  10  27  34  99]
 [245 244 242 238 234 229 224 220 216 212 212 158 1

frame before [[237 241 242 243 241 239 236 233 229 223 219 211 203 200 111  31  67  35
   27  25  20   7  19 144 156 157 155 153]
 [243 246 245 246 244 241 237 233 228 224 219 213 204 198 138  86  50  20
   28  28  24  21  36 155 158 154 152 151]
 [251 251 249 248 246 242 239 234 229 225 219 186 163 171 150 114  60  11
   17  12  10  21  67 159 158 155 155 150]
 [254 254 254 252 250 245 240 234 230 224 219 162 138 137 116 107  43   9
   32  10   8  25 104 160 157 154 152 151]
 [254 254 254 252 250 244 240 236 230 225 221 193 128 125 100 106  44  46
   61  27  21  37 106 159 155 153 151 149]
 [254 253 252 249 247 242 238 235 229 225 222 190 124 125 111 117  48  42
   41  25  35  41 129 159 155 151 148 147]
 [252 252 250 247 243 239 234 230 226 221 222 181 118 126 123 116  51  30
   12  17  29  39 138 157 155 153 149 147]
 [248 247 246 243 239 234 231 226 220 217 217 170 114 122 116 105  65  24
   13  10  23  39 138 155 155 153 152 149]
 [245 244 242 238 234 230 224 220 215 212 212 159 1

frame before [[237 240 242 243 242 239 236 232 228 223 204  39  63  42  30  29  24  13
   12 128 170 167 165 160 158 157 155 153]
 [242 245 245 246 244 241 237 233 228 223 210  60  87  30  30  30  21  24
   22 164 172 168 165 162 159 155 153 152]
 [252 251 249 249 246 242 239 233 229 225 217 113  96  19  17  17  11  25
   47 171 173 169 166 161 159 155 155 150]
 [254 254 253 252 250 245 239 234 230 224 219 145  88  19  28  25   8  23
   54 169 171 169 165 162 158 155 153 152]
 [254 254 254 252 249 245 240 236 230 224 221 165  93  39  76  44  31  41
   53 160 170 168 164 161 157 154 152 150]
 [253 253 252 249 248 242 239 234 229 225 222 162 107  44  39  21  30  36
   55 137 148 163 164 160 156 152 149 148]
 [252 252 250 247 243 239 234 230 226 220 221 165 110  45  30  17  20  31
   56 148 130 130 156 158 156 153 151 147]
 [248 247 245 243 239 234 231 226 220 217 217 167 105  55  31  18  14  28
   26  72 149 135 150 156 155 154 151 149]
 [245 243 242 238 235 229 224 219 215 212 211 160  

In [10]:
trainData = pd.read_csv('/home/deepak/Desktop/deepWork/machineLearning/dataset/mnistData/mnist_train.csv')
index = 4
testImg = trainData.iloc[index:(index+1), 1:]
cv2.imwrite("letter9.jpg",testImg.values.reshape(28, 28))

True

In [11]:
img = testImg.values.reshape(28, 28)
#print('testset', img)
path = 'letterOne.jpg'
predict(path)

predicted digit: 1


In [12]:
from PIL import Image 
image_file = Image.open("one.jpg") # open colour image
image_file = image_file.convert('1') # convert image to black and white
image_file.save('result.png')
